In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

1. Create a straight line dataset using the linear regression formula (weight * X + bias)
- Set `weight=0.3` and `bias=0.9` there should be at least 100 datapoints total
- Split the data into 80% training, 20% testing
- Plot the training and testing data so it becomes visual

In [ ]:
weight, bias = 0.3, 0.9
start, end, step = -1, 1, 0.02

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

print(f"Number of X samples: {len(X)}")
print(f"Number of y samples: {len(y)}")

split_training = int(0.8 * len(X))
X_train, X_test = X[:split_training], X[split_training:]
y_train, y_test = y[:split_training], y[split_training:]
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
def plot_predictions(train_data, train_labels, test_data, test_labels, predictions=None):
    """
    Plot training data, test data and compares predictions
    """
    plt.figure(figsize=(5,3))

    # Plot training data in blue
    plt.scatter(train_data, train_labels, c='b', s=4, label="Training data")

    # Plot test data in green
    plt.scatter(test_data, test_labels, c='g', s=4, label='Testing data')

    if predictions is not None:
        # Plot the predictions in red (predictions were made on the test data)
        plt.scatter(test_data, predictions, c='r', s=4, label='predictions')

    # Show the legend
    plt.legend(prop={'size': 14})

In [ ]:
plot_predictions(X_train, y_train, X_test, y_test)

2. Build a PyTorch model by subclassing `nn.Module`
- Inside should be a randomly initialized `nn.Parameter()` with `requires_grad=True`, one for `weights` and one for `bias`
- Implement the `forward()` method to compute the linear regression function you used to create the dataset in 1.
- Once you've constructed the model, make an instance of it and check its `state_dict()`.
- **Note** if you'd like to use `nn.Linear()` instead of `nn.Parameter()` you can.

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear_layer = nn.Linear(in_features=1, out_features=1, device=device, dtype=torch.float32)

    def forward(self, X):
        return self.linear_layer(X)

torch.manual_seed(42)
model = LinearRegressionModel()
model.state_dict()

3. Create a loss function and optimizer using `nn.L1Loss()` and `torch.optim.SGD(params, lr)` respectively
- Set the learning rate of the optimizer to be 0.01 and the parameters to optimize should be the model parameters from the model you created in 2.
- Write a training loop to perform the appropriate training steps for 300 epochs.
- The training loop should test the model on the test dataset every 20 epochs.

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

X_train, X_test = X_train.to(device), X_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)

epochs = 300

for epoch in range(epochs):
    ## Training

    model.train()

    # 1. Forward pass
    train_preds = model(X_train)

    # 2. Calculate loss
    train_loss = loss_fn(train_preds, y_train)

    # 3. Zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    train_loss.backward()

    # 5. Progress the optimizer
    optimizer.step()

    if epoch % 20 == 0:
    ## Test

        # put model in eval mode
        model.eval()

        with torch.inference_mode():

            # 1. Forward pass
            test_preds = model(X_test)

            # 2. Calculate loss
            test_loss = loss_fn(test_preds, y_test)

            print(f'epoch: {epoch} | train loss: {train_loss:.3f} | test loss: {test_loss:.3f}')


4. Make predictions with the trained model on the test data
- Visualize these predictions against the original training and testing data
(**note**: you may need to make sure the predictions are *not* on the GPU if you want to use non-CUDA-enabled libraries such as matplotlib to plot)

In [ ]:
model.eval()
with torch.inference_mode():
    y_preds = model(X_test)

plot_predictions(X_train.cpu(), y_train.cpu(), X_test.cpu(), y_test.cpu(), y_preds.cpu())

5. Save your trained model's `state_dict()` to file
- Create a new instance of your model class you made in 2, and load in the `state_dict()` you just saved to it.
- Perform predictions on your test data with the loaded model and confirm they match the original model predictions from 4.

In [ ]:
from pathlib import Path

# 1. Create models dir
MODEL_PATH = Path('models')
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = '01_pytorch_workflow_model_0.pth'
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
torch.save(model.state_dict(), MODEL_SAVE_PATH)

In [ ]:
loaded_model = LinearRegressionModel()
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH))

loaded_model.eval()
with torch.inference_mode():
    loaded_preds = loaded_model(X_test)

test_preds == loaded_preds